# 量子フーリエ変換を行列表示しよう

位相推定アルゴリズムの中で逆量子フーリエ変換が行われる。よってこの演算を行列表示する必要がある。

## やるべきことの確認
1. $R_{m}$ゲートを作る。
2. 第tビットを標的ビットとして、を作用させる演算を行列表示。($0\leqq t \leqq n-2$)  
第n-1ビットを標的ビットとするときは$H$ゲートのみを作用させる。
3. ビットを入れ替えるスワップゲートを行列表示
4. 最終的に求めた演算行列の転置複素共役をとって逆量子フーリエ変換の演算にする。

In [150]:
import numpy as np
import math

制御回転ゲートRmを作る。引数はビット数n,標的ビットの番号t,欲しいゲートm。なお制御ビットの番号はt+m-1で与えられる。  
**nは2の乗数であり、tは $0\leqq t \leqq n-2$ を満たし、mは $2\leqq m \leqq n-t$ を満たしている必要がある。**

In [151]:
def get_Rm(n,t,m):
    zero=np.array([[1,0],[0,0]])
    one=np.array([[0,0],[0,1]])
    
    Rm_zero=np.kron(np.eye((2**(t+m-1))),np.kron(zero,np.eye(2**(n-t-m))))
    Rm_one=np.kron(np.eye(2**t),np.kron(zero+(math.e**(2j*math.pi/2**m))*one,np.kron(np.eye((2**(m-2))),np.kron(one,np.eye((2**(n-t-m)))))))
    
    return Rm_zero+Rm_one

In [152]:
get_Rm(2,0,2)

array([[1.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 1.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 1.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        6.123234e-17+1.j]])

第tビット目を標的ビットにして$H$$R_{2}R_{3}$...$R_{n-t}$ゲートを作用させる演算を行列表示。($0\leqq t \leqq n-2$)  
ただし第n-1ビット目に対しては$H$ゲートのみを作用させる。

In [153]:
def get_HR(n,t):
    H=np.array([[1,1],[1,-1]])/math.sqrt(2)
    HR=np.kron(np.eye((2**t)),np.kron(H,np.eye(2**(n-t-1))))
    
    if t==n-1:
        return HR
    for i in range(2, n-t+1, 1):
        HR=get_Rm(n,t,i)@HR
    
    return HR

In [154]:
get_HR(2,0),get_HR(2,1)

(array([[ 7.07106781e-01+0.j        ,  0.00000000e+00+0.j        ,
          7.07106781e-01+0.j        ,  0.00000000e+00+0.j        ],
        [ 0.00000000e+00+0.j        ,  7.07106781e-01+0.j        ,
          0.00000000e+00+0.j        ,  7.07106781e-01+0.j        ],
        [ 7.07106781e-01+0.j        ,  0.00000000e+00+0.j        ,
         -7.07106781e-01+0.j        ,  0.00000000e+00+0.j        ],
        [ 0.00000000e+00+0.j        ,  4.32978028e-17+0.70710678j,
          0.00000000e+00+0.j        , -4.32978028e-17-0.70710678j]]),
 array([[ 0.70710678,  0.70710678,  0.        ,  0.        ],
        [ 0.70710678, -0.70710678,  0.        , -0.        ],
        [ 0.        ,  0.        ,  0.70710678,  0.70710678],
        [ 0.        , -0.        ,  0.70710678, -0.70710678]]))

スワップゲートを作用させることはは入れ替えたい2ビットa,bに対してCNOTゲートを三回作用させることと同じである。ただし1回目の制御ビットをaとすれば2回目の制御ビットはbで3回目はaでなければならない。

In [155]:
zero=np.array([[1,0],[0,0]])
one=np.array([[0,0],[0,1]])
X=np.array([[0,1],[1,0]])

def change(n,a,b):
    CNOT_a=np.kron(np.eye((2**a)),np.kron(zero,np.eye((2**(n-a-1)))))+np.kron(np.eye((2**a)),np.kron(one,np.kron(np.eye((2**(b-a-1))),np.kron(X,np.eye((2**(n-b-1)))))))
    CNOT_b=np.kron(np.eye((2**b)),np.kron(zero,np.eye((2**(n-b-1)))))+np.kron(np.eye((2**a)),np.kron(X,np.kron(np.eye((2**(b-a-1))),np.kron(one,np.eye((2**(n-b-1)))))))
    return CNOT_a@CNOT_b@CNOT_a

In [169]:
state=np.array([[0],[1],[0],[0]])
change(2,0,1)@state

array([[0.],
       [0.],
       [1.],
       [0.]])

0からn-1のビットをn/2-1番目とn/2番目を対称面として、すべて入れ替えるスワップゲートを行列表示する。

In [157]:
def get_swap(n):
    swap=np.eye((2**n))
    for i in range(int(n/2)):
        swap=change(n,i,n-1-i)@swap
    return swap

In [158]:
get_swap(2)

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

今までのを組み合わせて、量子フーリエ変換の演算を行列表示する。  
引数はビット数nのみ

In [159]:
def Q_Fourier(n):
    Q_Fourier=np.eye((2**n))
    for i in range(n):
        Q_Fourier=get_HR(n,i)@Q_Fourier
    return get_swap(n)@Q_Fourier

In [160]:
Q_Fourier(1) # 1ビットの量子フーリエ変換はHゲートに一致する

array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])

In [161]:
Q_Fourier(2)

array([[ 5.000000e-01+0.j ,  5.000000e-01+0.j ,  5.000000e-01+0.j ,
         5.000000e-01+0.j ],
       [ 5.000000e-01+0.j ,  3.061617e-17+0.5j, -5.000000e-01+0.j ,
        -3.061617e-17-0.5j],
       [ 5.000000e-01+0.j , -5.000000e-01+0.j ,  5.000000e-01+0.j ,
        -5.000000e-01+0.j ],
       [ 5.000000e-01+0.j , -3.061617e-17-0.5j, -5.000000e-01+0.j ,
         3.061617e-17+0.5j]])

逆量子フーリエ変換の行列表示を返すメソッド

In [162]:
def inv_Fourier(n):
    IF=np.conjugate(Q_Fourier(n))
    return IF.T

In [163]:
inv_Fourier(1) @ Q_Fourier(1)

array([[1., 0.],
       [0., 1.]])